In [1]:
import os
import polars as pl

os.chdir(os.getcwd())

In [2]:
ridership_df = pl.read_csv('data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals_20250203.csv')

In [3]:
ridership_df.head(10)

station_id,stationname,date,daytype,rides
i64,str,str,str,i64
41280,"""Jefferson Park""","""12/22/2017""","""W""",6104
41000,"""Cermak-Chinatown""","""12/18/2017""","""W""",3636
40280,"""Central-Lake""","""12/02/2017""","""A""",1270
40140,"""Dempster-Skokie""","""12/19/2017""","""W""",1759
40690,"""Dempster""","""12/03/2017""","""U""",499
41660,"""Lake/State""","""12/30/2017""","""A""",8615
40180,"""Oak Park-Forest Park""","""12/17/2017""","""U""",442
40250,"""Kedzie-Homan-Forest Park""","""12/02/2017""","""A""",1353
40120,"""35th/Archer""","""12/07/2017""","""W""",3353


In [7]:
# Generate int columns for year, month, and day from the date column
ridership_df = ridership_df.with_columns(
    pl.col('date').str.to_date('%m/%d/%Y')
)

ridership_df = ridership_df.with_columns(
    year=pl.col('date').dt.year(),
    month=pl.col('date').dt.month(),
    day=pl.col('date').dt.day()
)

SchemaError: invalid series dtype: expected `String`, got `date` for series with name `date`

In [14]:
# Filter to a restricted date range to limit the size of the data in memory
min_year = 2018
max_year = 2024

ridership_df = ridership_df.filter(
    pl.col('year').is_between(min_year, max_year)
)

ridership_df = ridership_df.sort(['station_id', 'year', 'month', 'day'])

In [15]:
ridership_df.head(10)

station_id,stationname,date,daytype,rides,year,month,day
i64,str,datetime[μs],str,i64,i32,i8,i8
40010,"""Austin-Forest Park""",2018-01-01 00:00:00,"""U""",343,2018,1,1
40010,"""Austin-Forest Park""",2018-01-02 00:00:00,"""W""",1187,2018,1,2
40010,"""Austin-Forest Park""",2018-01-03 00:00:00,"""W""",1429,2018,1,3
40010,"""Austin-Forest Park""",2018-01-04 00:00:00,"""W""",1447,2018,1,4
40010,"""Austin-Forest Park""",2018-01-05 00:00:00,"""W""",1391,2018,1,5
40010,"""Austin-Forest Park""",2018-01-06 00:00:00,"""A""",532,2018,1,6
40010,"""Austin-Forest Park""",2018-01-07 00:00:00,"""U""",450,2018,1,7
40010,"""Austin-Forest Park""",2018-01-08 00:00:00,"""W""",1773,2018,1,8
40010,"""Austin-Forest Park""",2018-01-09 00:00:00,"""W""",1808,2018,1,9


In [ ]:
# Todo: check that station ID and date are unique
# Left join with station ID and names from system information dataset